In [1]:
import tensorflow as tf
import numpy as np

## test the ion representation model

In [2]:
batch_size = 2
num_ions = 2
hidden_size = 3
M = 5

In [3]:
hr_np = np.arange(batch_size * 1 * num_ions * hidden_size).reshape(batch_size,1,num_ions,hidden_size).astype(np.float32)
location_np = np.array([[0,4], [2, 4]]).astype(np.int32).reshape(2,1,2)

In [4]:
location_index = tf.placeholder(tf.int32, (batch_size, 1, num_ions), name='location')
#ion_hidden_represent = tf.placeholder(tf.float32, (batch_size, 1, num_ions, hidden_size), name='hidden_representation')
ion_hidden_represent = tf.constant(hr_np)
resized_ion_hidden = tf.reshape(ion_hidden_represent, (-1, num_ions, hidden_size))
resized_location_index = tf.reshape(location_index, (-1, num_ions))

In [9]:
output = tf.Variable(tf.zeros((batch_size, M, hidden_size), dtype=tf.float32), trainable=False, collections=['not_saved'])
# make sure in before each forward step output is reset to 0
assign_op = tf.assign(output, tf.zeros((batch_size, M, hidden_size)))


with tf.control_dependencies([tf.identity(assign_op), tf.identity(resized_location_index)]):
    gathered_output = tf.batch_gather(params=output, indices=resized_location_index) #(batch, resize_location_index, hidden_size)
    update = gathered_output + resized_ion_hidden
    output = tf.batch_scatter_update(ref=output, indices=resized_location_index, updates=update, name='batch_scatter_update')
final_output = tf.reduce_mean(output)

In [15]:
with tf.Session() as sess:
    init_op = tf.initializers.variables(tf.global_variables() + tf.get_collection('not_saved'))
    sess.run(init_op)
    #print(sess.run(gathered_output, feed_dict={location_index:location_np}))
    print(sess.run(final_output, feed_dict={location_index:location_np}))
    print(sess.run(final_output, feed_dict={location_index:location_np}))
    print(sess.run(output, feed_dict={location_index:location_np}))

2.2
2.2
[[[ 0.  1.  2.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 3.  4.  5.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 6.  7.  8.]
  [ 0.  0.  0.]
  [ 9. 10. 11.]]]


In [ ]:
ion_hidden_represent